In [1]:
:set -XNoImplicitPrelude

In [2]:
module TreeUnfold (unfoldFullTree) where

import Papa
import Data.Tree.Lens
import Data.Tree

-- check if the root of tree matches to the incoming item
check :: Eq a => a -> Tree a -> Bool
check i = (== i) . rootLabel

-- create a traversal of plates as per the depth required by the unfold operation
platesPerDepth :: (Plated a, Applicative f) => Int -> ((a -> f a) -> a -> f a)
platesPerDepth i = foldl' (\g p -> plate . g) id [0..i]

fn :: Eq a => Int -> (Tree a, [a]) -> a -> (Tree a, [a])
fn i (ts,ys) x = if isInLevel 
                then 
                    (ts,ys) -- ^ if new node is already there then move on
                else 
                    (append ts,ys) -- ^ append the new node to the right parent
    where
        newNode = Node x [] -- ^ new node
        parent = ys ^?! ix (i - 1) -- ^ parent index, required in case of a new child
        isInLevel = isJust $ ts ^? platesPerDepth i . filtered (check x) -- ^ check if new node is already there in this level
        append = bool (over (platesPerDepth (i - 1) . filtered (check parent)) (branches <>~ [newNode])) -- ^ add the new node as a new child to a specific parent
                      (over branches (<> [newNode])) -- ^ if first node then just add it there
                      (i == 0) -- ^ check if it's first node


-- unfold a tree from list of lists where each list is for items that signifies
-- the path of an element (last one of the list) right from the top of the tree
-- 
-- the function takes an initial tree as the container tree within which rest of the final tree is constructed
-- 
unfoldFullTree :: Eq a => Tree a -> [[a]] -> Tree a
unfoldFullTree = foldl' (\t ys -> fst $  ifoldl' fn (t,ys) ys)


In [5]:
-- Tests
import Papa
import Data.Tree

tr1 = Node 0 []

inp = [[1,2,4],[1,2],[1,2,4,5],[1],[1,2,3]]
unfoldFullTree tr1 inp == Node 0 [Node 1 [Node 2 [Node 4 [Node 5 []], Node 3 []]]]

inp1 = [[1],[2],[3]]
unfoldFullTree tr1 inp1 == Node 0 [Node 1 [], Node 2 [], Node 3 []]

inp2 = [[1,2,3], [2,3,4], [1,4,6]]
unfoldFullTree tr1 inp2 == Node 0 [Node 1 [Node 2 [Node 3 []], Node 4 [Node 6 []]], Node 2 [Node 3 [Node 4 []]]]

inp3 = [[1,2],[1,3],[1,4],[1],[1,3,5]]
unfoldFullTree tr1 inp3 == Node 0 [Node 1 [Node 2 [], Node 3 [Node 5 []], Node 4 []]]

inp4 = [[1,3],[1,2,5],[1,3,6,7],[1],[1,2]]
unfoldFullTree tr1 inp4 == Node 0 [Node 1 [Node 3 [Node 6 [Node 7 []]], Node 2 [Node 5 []]]]

inp5 = [[1,2,3],[1,2],[1,3,2],[1,3],[1]]
unfoldFullTree tr1 inp5 == Node 0 [Node 1 [Node 2 [Node 3 []], Node 3 [Node 2 []]]]





True

True

True

True

True

True